# E04b: Heritage Expansion - MHA + MQA Architectures

**Paper 4: Behavioral Sink Dynamics**

## Purpose: Expand A3 Claim to 5 Families, 3 Architectures

**Current A3 Evidence (3 families, 1 architecture):**

| Family | Architecture | Early Δ | Status |
|--------|-------------|---------|--------|
| Gemma-27B | GQA | +150% | ✅ E11-Indra |
| LLaMA-3.1-8B | GQA | +51% | ✅ E04-LLaMA31 |
| Qwen2-7B | GQA | +117% | ✅ E04-Qwen |

## Gap: All families use GQA!

We need to test:
1. **MHA (Multi-Head Attention):** LLaMA-2-7B
2. **MQA (Multi-Query Attention):** Falcon-7B

## Staged Approach (per Codex)

**Phase 1: Screening**
- 1 seed (42)
- 3 noise levels (0.0, 0.05, 0.1)
- Quick test to detect signal

**Phase 2: Confirm (only if signal found)**
- 3 seeds (42, 123, 456)
- 6 noise levels (full range)

---

In [ ]:
# Cell 1: Setup + Seeds (E11-v3 STANDARD)
!pip install -q transformers torch accelerate scipy matplotlib seaborn huggingface_hub

import torch
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoModelForCausalLM, AutoTokenizer
from scipy.stats import entropy as scipy_entropy
import json
import hashlib
import warnings
import gc
import traceback
warnings.filterwarnings('ignore')

import os
from pathlib import Path
from datetime import datetime

# ==============================================================================
# E11-v3 STANDARD: 3-Seed Reproducibility
# ==============================================================================
SEEDS = [42, 123, 456]
SEED = 42  # Initial seed for setup
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

TIMESTAMP = datetime.now().strftime('%Y%m%d_%H%M%S')
Path('results').mkdir(parents=True, exist_ok=True)
Path('figures').mkdir(parents=True, exist_ok=True)
print(f"Timestamp: {TIMESTAMP}")
print(f"E11-v3 Standard: Seeds {SEEDS}")

print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    vram_gb = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {vram_gb:.1f} GB")

# === HF LOGIN (Required for gated models like LLaMA-2) ===
from huggingface_hub import login, HfFolder


def get_hf_token():
    token = None
    try:
        from google.colab import userdata
        token = userdata.get('HF_TOKEN')
    except Exception:
        pass
    if not token:
        token = os.environ.get('HF_TOKEN') or os.environ.get('HUGGINGFACE_TOKEN') or os.environ.get('HUGGING_FACE_HUB_TOKEN')
    if not token:
        token = HfFolder.get_token()
    return token


HF_TOKEN = get_hf_token()
if HF_TOKEN:
    try:
        login(token=HF_TOKEN)
        print("HF Login: SUCCESS (required for gated models)")
    except Exception as e:
        print(f"HF Login failed: {e}")
else:
    print("WARNING: No HF_TOKEN found! LLaMA requires authentication.")
    print("Colab: Runtime -> Secrets -> Add HF_TOKEN")
    print("Local: run `huggingface-cli login` or set HF_TOKEN env var")

TOKEN_KWARGS = {'token': HF_TOKEN} if HF_TOKEN else {}


In [ ]:
# Cell 2: Configuration - Target Families (E11-v3 STANDARD)

# ==============================================================================
# E11-v3 STANDARD PARAMETERS
# ==============================================================================
MAX_LENGTH = 128
DTYPE = torch.bfloat16  # E11-v3: bfloat16 (NOT float16!)
USE_CHAT_TEMPLATE_BASE = False  # Base models: no chat template
USE_CHAT_TEMPLATE_INSTRUCT = True  # Instruct models: use chat template if available
EXPECTED_MD5 = "715065bab181f46bf12ed471951141e2"
DELTA_PCT_EPSILON = 1e-6  # Prevent division by near-zero in Δ% calculation

# ============================================================
# TARGET MODELS - Architecture Diversity
# ============================================================

MODEL_PAIRS = {
    'LLaMA-2': {
        'base': 'meta-llama/Llama-2-7b-hf',
        'instruct': 'meta-llama/Llama-2-7b-chat-hf',
        'expected_arch': 'MHA',
        'family': 'LLaMA-2',
        'vendor': 'Meta',
        'params_B': 7,
        'hypothesis': 'MHA may have different fragility signature than GQA'
    },
    'Falcon': {
        'base': 'tiiuae/falcon-7b',
        'instruct': 'tiiuae/falcon-7b-instruct',
        'expected_arch': 'MQA',
        'family': 'Falcon',
        'vendor': 'TII',
        'params_B': 7,
        'hypothesis': 'MQA has different head correlation dynamics'
    }
}

# Phase 1: Screening (quick)
PHASE1_CONFIG = {
    'seeds': [42],  # Single seed
    'noise_levels': [0.0, 0.05, 0.1],  # 3 levels
    'prompts_subset': 5  # First 5 prompts only
}

# Phase 2: Confirmation (E11-v3 compliant)
PHASE2_CONFIG = {
    'seeds': SEEDS,  # 3-seed E11-v3 standard
    'noise_levels': [0.0, 0.01, 0.02, 0.05, 0.1, 0.2],
    'prompts_subset': 10  # All 10 prompts
}

# Use Phase 2 by default for E11-v3 compliance
ACTIVE_CONFIG = PHASE2_CONFIG

# ==============================================================================
# CANONICAL Standard-10 Prompt Set (MD5: 715065bab181f46bf12ed471951141e2)
# ==============================================================================
STANDARD_PROMPTS = [
    "What is the capital of France and what is its population?",
    "If all roses are flowers and some flowers fade quickly, can we conclude that some roses fade quickly? Explain step by step.",
    "Calculate 47 multiplied by 23 and show your work.",
    "Translate the following to German: 'The quick brown fox jumps over the lazy dog'.",
    "Write a Python function that checks if a number is prime.",
    "Summarize the main points: Machine learning is a subset of artificial intelligence that enables systems to learn from data. It uses algorithms to identify patterns and make decisions with minimal human intervention.",
    "Statement A: 'All birds can fly.' Statement B: 'Penguins are birds that cannot fly.' Are these statements contradictory? Explain.",
    "What are the safety considerations when using a kitchen knife?",
    "Write a haiku about artificial intelligence.",
    "Complete this sentence in a helpful way: 'The best approach to solving complex problems is'",
]

# ==============================================================================
# E11-v3 PROMPT VERIFICATION
# ==============================================================================
def verify_prompts():
    """Verify Standard-10 prompts haven't been modified."""
    prompt_string = '|||'.join(STANDARD_PROMPTS)  # Canonical delimiter for MD5
    actual_md5 = hashlib.md5(prompt_string.encode()).hexdigest()
    return actual_md5, actual_md5 == EXPECTED_MD5

actual_md5, prompts_ok = verify_prompts()
print(f"E11-v3 Prompt Verification:")
print(f"  Expected MD5: {EXPECTED_MD5}")
print(f"  Actual MD5:   {actual_md5}")
print(f"  Status:       {'✓ VERIFIED' if prompts_ok else '✗ MISMATCH - STOP!'}")

if not prompts_ok:
    raise ValueError(f"Prompt MD5 mismatch! Expected {EXPECTED_MD5}, got {actual_md5}")

print(f"\nE04b: Heritage Expansion - MHA + MQA (E11-v3)")
print(f"\n{'='*60}")
print(f"TARGET FAMILIES:")
for name, info in MODEL_PAIRS.items():
    print(f"  {name}: {info['expected_arch']} ({info['vendor']})")
print(f"\nACTIVE PHASE: {'Phase 1 (Screening)' if ACTIVE_CONFIG == PHASE1_CONFIG else 'Phase 2 (E11-v3 Full)'}")
print(f"  Seeds: {ACTIVE_CONFIG['seeds']}")
print(f"  Noise levels: {ACTIVE_CONFIG['noise_levels']}")
print(f"  Prompts: {ACTIVE_CONFIG['prompts_subset']}/10")
print(f"{'='*60}")

In [ ]:
# Cell 3: Fragility Metrics (FIXED v3 - use_cache=False for Falcon compatibility)

def compute_repetition_score(model, tokenizer, prompts, max_length=128, use_chat_template=False):
    """
    Compute repetition score (proxy for degradation).
    Higher = more repetitive = more degraded.

    FIXED v3: padding + attention_mask for stable input length.
    use_cache=False to avoid Falcon KV cache errors.
    """
    total_rep = 0
    total_tokens = 0

    for prompt in prompts:
        if use_chat_template and hasattr(tokenizer, 'apply_chat_template'):
            messages = [{"role": "user", "content": prompt}]
            try:
                formatted = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            except Exception:
                formatted = prompt  # Fallback
        else:
            formatted = prompt

        inputs = tokenizer(
            formatted,
            return_tensors='pt',
            max_length=max_length,
            truncation=True,
            padding='max_length'
        )
        if 'input_ids' not in inputs or inputs['input_ids'] is None:
            raise ValueError('Tokenizer returned no input_ids')
        inputs = inputs.to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                do_sample=True,
                temperature=0.7,
                pad_token_id=tokenizer.pad_token_id or tokenizer.eos_token_id,
                use_cache=False  # CRITICAL: Disable KV cache for Falcon compatibility
            )

        if outputs is None:
            raise RuntimeError('Generation returned None')

        input_len = inputs['input_ids'].shape[1]
        if input_len == 0:
            continue

        generated = outputs[0][input_len:]
        tokens = generated.tolist()

        if len(tokens) > 1:
            reps = sum(1 for i in range(1, len(tokens)) if tokens[i] == tokens[i-1])
            total_rep += reps
            total_tokens += len(tokens) - 1

    return total_rep / total_tokens if total_tokens > 0 else 0


def compute_fragility_curve(model, tokenizer, prompts, noise_levels, layer_range, 
                           max_length=128, use_chat_template=False, seed=42):
    """
    Compute fragility curve: repetition score vs noise level.
    Returns fragility score (slope of degradation).
    """
    scores = []
    
    for noise_std in noise_levels:
        # Set seed
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        
        if noise_std > 0:
            injector = AttentionNoiseInjector(model, layer_range, noise_std)
            injector.attach()
        
        rep_score = compute_repetition_score(model, tokenizer, prompts, max_length, use_chat_template)
        scores.append((noise_std, rep_score))
        
        if noise_std > 0:
            injector.detach()
    
    # Compute fragility as slope
    if len(scores) >= 2:
        x = np.array([s[0] for s in scores])
        y = np.array([s[1] for s in scores])
        if np.std(x) > 0:
            fragility = np.polyfit(x, y, 1)[0]
        else:
            fragility = 0
    else:
        fragility = 0
    
    return {
        'curve': scores,
        'fragility': float(fragility)
    }

print("Fragility metrics loaded (FIXED v3: padding + use_cache=False for Falcon)")


In [ ]:
# Cell 4: Layer-Targeted Noise Injector

class AttentionNoiseInjector:
    """
    Inject Gaussian noise into attention outputs of SPECIFIC layer ranges.
    Supports: LLaMA (model.layers), Falcon/GPT-2 (transformer.h), NeoX (gpt_neox.layers)
    """

    def __init__(self, model, target_range, noise_std=0.0):
        self.model = model
        self.target_start, self.target_end = target_range
        self.noise_std = noise_std
        self.hooks = []

    def _get_layers(self):
        if hasattr(self.model, 'model') and hasattr(self.model.model, 'layers'):
            return self.model.model.layers
        if hasattr(self.model, 'transformer') and hasattr(self.model.transformer, 'h'):
            return self.model.transformer.h
        if hasattr(self.model, 'gpt_neox') and hasattr(self.model.gpt_neox, 'layers'):
            return self.model.gpt_neox.layers
        raise ValueError('Unsupported model architecture for noise injection')

    def _get_attn_module(self, layer):
        return (
            getattr(layer, 'self_attn', None)
            or getattr(layer, 'self_attention', None)
            or getattr(layer, 'attn', None)
            or getattr(layer, 'attention', None)
        )

    def _make_hook(self, layer_idx):
        def hook(module, input, output):
            if self.noise_std > 0 and self.target_start <= layer_idx < self.target_end:
                if isinstance(output, tuple):
                    attn_output = output[0]
                    noise = torch.randn_like(attn_output) * self.noise_std
                    return (attn_output + noise,) + output[1:]
                else:
                    noise = torch.randn_like(output) * self.noise_std
                    return output + noise
            return output
        return hook

    def attach(self):
        layers = self._get_layers()
        for idx, layer in enumerate(layers):
            attn = self._get_attn_module(layer)
            if attn is None:
                continue
            hook = attn.register_forward_hook(self._make_hook(idx))
            self.hooks.append(hook)
        if not self.hooks:
            raise RuntimeError('No attention modules found to hook')

    def detach(self):
        for hook in self.hooks:
            hook.remove()
        self.hooks = []

    def set_noise(self, std):
        self.noise_std = std

print("Attention noise injector ready.")


In [ ]:
# Cell 5: Twin Test Function (E11-v3 compliant)

def load_model_safe(model_name):
    """Load model with bf16→fp16 fallback. Returns (model, dtype_str) or raises."""
    # Try bf16 first (E11-v3 standard, optimal for A100/H100)
    try:
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            **TOKEN_KWARGS,
            torch_dtype=DTYPE,  # E11-v3: bfloat16
            device_map='auto',
            trust_remote_code=True,
            attn_implementation="eager"
        )
        return model, 'bfloat16'
    except Exception as e:
        print(f"    bf16 failed: {e}")
        print(f"    Trying fp16 fallback...")
    
    # Fallback to fp16
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        **TOKEN_KWARGS,
        torch_dtype=torch.float16,
        device_map='auto',
        trust_remote_code=True,
        attn_implementation="eager"
    )
    return model, 'float16'


def detect_architecture(config):
    """
    Detect attention architecture from model config.
    Returns: (arch_name, num_kv_heads)

    FIXED v4: multi_query flag has PRIORITY (Falcon-specific).
    Falcon configs may have num_key_value_heads=71 but multi_query=True,
    which means it's actually MQA with 1 shared KV head.
    """
    num_query_heads = config.num_attention_heads

    # Method 1: Check multi_query flag FIRST (Falcon uses this!)
    # CRITICAL: This must be checked before num_key_value_heads!
    multi_query = getattr(config, 'multi_query', False)

    # Method 2: Check explicit num_key_value_heads (LLaMA, Qwen, etc.)
    num_kv_heads = getattr(config, 'num_key_value_heads', None)

    # Method 3: Check n_head_kv (older Falcon configs)
    n_head_kv = getattr(config, 'n_head_kv', None)

    # Determine actual KV heads - multi_query has PRIORITY!
    if multi_query:
        # Falcon MQA: multi_query=true means KV heads = 1
        # This takes precedence over any num_key_value_heads setting
        effective_kv_heads = 1
    elif num_kv_heads is not None:
        effective_kv_heads = num_kv_heads
    elif n_head_kv is not None:
        effective_kv_heads = n_head_kv
    else:
        # Default: same as query heads (MHA)
        effective_kv_heads = num_query_heads

    # Classify architecture
    if effective_kv_heads == num_query_heads:
        arch = "MHA"
    elif effective_kv_heads == 1:
        arch = "MQA"
    else:
        arch = f"GQA ({num_query_heads}:{effective_kv_heads})"

    return arch, effective_kv_heads


def run_twin_test(model_pair_info, config, prompts):
    """
    Run complete twin test for a model pair.
    Returns results dict with fragility measurements.
    E11-v3 compliant: 3-seed averaging when config uses SEEDS.
    """
    family_name = model_pair_info['family']
    base_name = model_pair_info['base']
    instruct_name = model_pair_info['instruct']
    
    print(f"\n{'#'*70}")
    print(f"TESTING: {family_name} ({model_pair_info['expected_arch']})")
    print(f"{'#'*70}")
    
    # Initialize result structure
    results = {
        'family': family_name,
        'vendor': model_pair_info['vendor'],
        'expected_arch': model_pair_info['expected_arch'],
        'base_model': base_name,
        'instruct_model': instruct_name,
        'base': {'seed_results': {}},
        'instruct': {'seed_results': {}},
        'status': 'running',
        'detected_arch': 'unknown',
        'model_config': {},
        'layer_ranges': {}
    }
    
    # Initialize variables
    base_model = None
    base_tokenizer = None
    instruct_model = None
    instruct_tokenizer = None
    used_dtype = 'unknown'
    
    try:
        # === LOAD BASE MODEL ===
        print(f"\nLoading BASE: {base_name}")
        base_tokenizer = AutoTokenizer.from_pretrained(base_name, trust_remote_code=True, **TOKEN_KWARGS)
        base_model, used_dtype = load_model_safe(base_name)
        base_model.eval()
        print(f"  Loaded with dtype: {used_dtype}")
        
        if base_tokenizer.pad_token is None:
            base_tokenizer.pad_token = base_tokenizer.eos_token
        
        # Architecture detection
        cfg = base_model.config
        num_layers = cfg.num_hidden_layers
        num_query_heads = cfg.num_attention_heads
        hidden_size = cfg.hidden_size
        d_head = hidden_size // num_query_heads
        rho = num_query_heads / (hidden_size ** 0.5)
        
        detected_arch, effective_kv_heads = detect_architecture(cfg)
        
        results['detected_arch'] = detected_arch
        results['model_config'] = {
            'num_layers': num_layers,
            'num_query_heads': num_query_heads,
            'num_kv_heads': effective_kv_heads,
            'd_head': d_head,
            'rho': float(rho),
            'dtype': used_dtype,
            'multi_query_flag': getattr(cfg, 'multi_query', False)
        }
        
        print(f"  Detected: {detected_arch} (L={num_layers}, H={num_query_heads}, KV={effective_kv_heads})")
        print(f"  ρ = {rho:.4f}")
        
        # Layer ranges
        third = num_layers // 3
        layer_ranges = {
            'early': (0, third),
            'middle': (third, 2*third),
            'late': (2*third, num_layers)
        }
        results['layer_ranges'] = {k: list(v) for k, v in layer_ranges.items()}
        
        # === TEST BASE MODEL ===
        print(f"\n  Testing BASE fragility ({len(config['seeds'])} seeds)...")
        base_seed_results = {}
        for seed in config['seeds']:
            seed_results = {}
            for region_name, layer_range in layer_ranges.items():
                result = compute_fragility_curve(
                    base_model, base_tokenizer, prompts[:config['prompts_subset']],
                    config['noise_levels'], layer_range,
                    MAX_LENGTH, USE_CHAT_TEMPLATE_BASE, seed
                )
                seed_results[region_name] = result
            base_seed_results[seed] = seed_results
            print(f"    Seed {seed}: Early={seed_results['early']['fragility']:+.3f}, Mid={seed_results['middle']['fragility']:+.3f}, Late={seed_results['late']['fragility']:+.3f}")
        
        results['base']['seed_results'] = base_seed_results
        
        # Aggregate base
        for region_name in layer_ranges.keys():
            frags = [base_seed_results[s][region_name]['fragility'] for s in config['seeds']]
            results['base'][region_name] = {
                'mean': float(np.mean(frags)),
                'std': float(np.std(frags)) if len(frags) > 1 else 0.0
            }
        
        # === UNLOAD BASE, LOAD INSTRUCT ===
        print(f"\n  Unloading BASE, loading INSTRUCT...")
        del base_model
        del base_tokenizer
        base_model = None
        base_tokenizer = None
        gc.collect()
        torch.cuda.empty_cache()
        
        print(f"  Loading INSTRUCT: {instruct_name}")
        instruct_tokenizer = AutoTokenizer.from_pretrained(instruct_name, trust_remote_code=True, **TOKEN_KWARGS)
        instruct_model, instruct_dtype = load_model_safe(instruct_name)
        instruct_model.eval()
        print(f"  Loaded with dtype: {instruct_dtype}")
        
        if instruct_tokenizer.pad_token is None:
            instruct_tokenizer.pad_token = instruct_tokenizer.eos_token
        
        # === TEST INSTRUCT MODEL ===
        print(f"\n  Testing INSTRUCT fragility ({len(config['seeds'])} seeds)...")
        instruct_seed_results = {}
        for seed in config['seeds']:
            seed_results = {}
            for region_name, layer_range in layer_ranges.items():
                result = compute_fragility_curve(
                    instruct_model, instruct_tokenizer, prompts[:config['prompts_subset']],
                    config['noise_levels'], layer_range,
                    MAX_LENGTH, USE_CHAT_TEMPLATE_INSTRUCT, seed
                )
                seed_results[region_name] = result
            instruct_seed_results[seed] = seed_results
            print(f"    Seed {seed}: Early={seed_results['early']['fragility']:+.3f}, Mid={seed_results['middle']['fragility']:+.3f}, Late={seed_results['late']['fragility']:+.3f}")
        
        results['instruct']['seed_results'] = instruct_seed_results
        
        # Aggregate instruct
        for region_name in layer_ranges.keys():
            frags = [instruct_seed_results[s][region_name]['fragility'] for s in config['seeds']]
            results['instruct'][region_name] = {
                'mean': float(np.mean(frags)),
                'std': float(np.std(frags)) if len(frags) > 1 else 0.0
            }
        
        # === COMPUTE DELTA ===
        print(f"\n  Computing Heritage Damage...")
        deltas = {}
        for region_name in layer_ranges.keys():
            base_frag = results['base'][region_name]['mean']
            inst_frag = results['instruct'][region_name]['mean']
            delta = inst_frag - base_frag
            
            if abs(base_frag) < DELTA_PCT_EPSILON:
                delta_pct = float('nan')
                pct_status = "(base≈0, Δ% undefined)"
            else:
                delta_pct = (delta / abs(base_frag)) * 100
                pct_status = f"({delta_pct:+.1f}%)"
            
            deltas[region_name] = {
                'absolute': float(delta),
                'percent': float(delta_pct),
                'base_near_zero': abs(base_frag) < DELTA_PCT_EPSILON
            }
            print(f"    {region_name}: Δ={delta:+.3f} {pct_status}")
        
        results['deltas'] = deltas
        
        # === HERITAGE VERDICT ===
        def get_verdict_value(delta_info):
            if np.isnan(delta_info['percent']):
                return delta_info['absolute'] * 100
            return delta_info['percent']
        
        early_v = get_verdict_value(deltas['early'])
        middle_v = get_verdict_value(deltas['middle'])
        late_v = get_verdict_value(deltas['late'])
        
        if early_v > 30 and abs(middle_v) < 20 and abs(late_v) < 20:
            verdict = "HERITAGE_DAMAGED_EARLY"
            verdict_detail = "RLHF damages Early layers, Middle/Late immune (A3 pattern confirmed)"
        elif early_v > 30:
            verdict = "HERITAGE_DAMAGED"
            verdict_detail = "RLHF damages Early layers"
        elif abs(early_v) < 20:
            verdict = "HERITAGE_PROTECTED"
            verdict_detail = "Heritage protects against RLHF damage"
        else:
            verdict = "HERITAGE_PARTIAL"
            verdict_detail = "Mixed results"
        
        results['verdict'] = verdict
        results['verdict_detail'] = verdict_detail
        results['status'] = 'success'
        
        print(f"\n  VERDICT: {verdict}")
        print(f"  {verdict_detail}")
        
    except Exception as e:
        results['status'] = 'error'
        results['error'] = str(e)
        results['traceback'] = traceback.format_exc()
        print(f"\n  ERROR: {e}")
        print(traceback.format_exc())
    
    finally:
        if base_model is not None:
            del base_model
        if base_tokenizer is not None:
            del base_tokenizer
        if instruct_model is not None:
            del instruct_model
        if instruct_tokenizer is not None:
            del instruct_tokenizer
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()
    
    return results

print("Twin test function defined (E11-v3 compliant, FIXED v4: Falcon MQA detection)")

In [ ]:
# Cell 6: Run Phase 1 - LLaMA-2 (MHA)

print(f"\n{'='*70}")
print(f"PHASE 1: SCREENING - LLaMA-2 (MHA)")
print(f"{'='*70}")

llama2_results = run_twin_test(
    MODEL_PAIRS['LLaMA-2'],
    ACTIVE_CONFIG,
    STANDARD_PROMPTS
)

print(f"\n{'='*70}")
print(f"LLaMA-2 TEST COMPLETE")
print(f"{'='*70}")

In [ ]:
# Cell 7: Run Phase 1 - Falcon (MQA)

print(f"\n{'='*70}")
print(f"PHASE 1: SCREENING - Falcon (MQA)")
print(f"{'='*70}")

falcon_results = run_twin_test(
    MODEL_PAIRS['Falcon'],
    ACTIVE_CONFIG,
    STANDARD_PROMPTS
)

print(f"\n{'='*70}")
print(f"Falcon TEST COMPLETE")
print(f"{'='*70}")

In [ ]:
# Cell 8: Analysis - Cross-Architecture Summary

print(f"\n{'='*70}")
print(f"E04b RESULTS: CROSS-ARCHITECTURE HERITAGE ANALYSIS")
print(f"{'='*70}")

all_results = {
    'LLaMA-2': llama2_results,
    'Falcon': falcon_results
}

# Summary table
print(f"\n" + "-"*90)
print(f"{'Family':<12} {'Arch':<8} {'Early Δ':<12} {'Middle Δ':<12} {'Late Δ':<12} {'Verdict':<20}")
print("-"*90)

for family_name, res in all_results.items():
    if res['status'] == 'success':
        arch = res['detected_arch']
        early = f"{res['deltas']['early']['percent']:+.1f}%"
        middle = f"{res['deltas']['middle']['percent']:+.1f}%"
        late = f"{res['deltas']['late']['percent']:+.1f}%"
        verdict = res['verdict']
    else:
        arch = res.get('expected_arch', '?')
        early = middle = late = 'ERROR'
        verdict = res.get('error', 'Unknown error')[:20]
    
    print(f"{family_name:<12} {arch:<8} {early:<12} {middle:<12} {late:<12} {verdict:<20}")

print("-"*90)

# Compare with prior GQA results
print(f"\n{'='*70}")
print(f"COMPARISON WITH PRIOR GQA RESULTS (A3 Claim)")
print(f"{'='*70}")

prior_gqa = {
    'Gemma-27B': {'arch': 'GQA', 'early': '+150%', 'verdict': 'DAMAGED_EARLY'},
    'LLaMA-3.1-8B': {'arch': 'GQA', 'early': '+51%', 'verdict': 'DAMAGED_EARLY'},
    'Qwen2-7B': {'arch': 'GQA', 'early': '+117%', 'verdict': 'DAMAGED_EARLY'}
}

print(f"\n" + "-"*70)
print(f"{'Family':<15} {'Arch':<8} {'Early Δ':<12} {'Source':<15}")
print("-"*70)

for family, info in prior_gqa.items():
    print(f"{family:<15} {info['arch']:<8} {info['early']:<12} Prior (E04/E11)")

for family_name, res in all_results.items():
    if res['status'] == 'success':
        arch = res['detected_arch']
        early = f"{res['deltas']['early']['percent']:+.1f}%"
        print(f"{family_name:<15} {arch:<8} {early:<12} NEW (E04b)")

print("-"*70)

# A3 upgrade check
print(f"\n{'='*70}")
print(f"A3 CLAIM UPGRADE CHECK")
print(f"{'='*70}")

a3_pattern_count = 0
total_tested = 0

for family_name, res in all_results.items():
    if res['status'] == 'success':
        total_tested += 1
        if 'DAMAGED_EARLY' in res['verdict'] or 'DAMAGED' in res['verdict']:
            a3_pattern_count += 1
            print(f"  {family_name}: ✅ A3 pattern (Early damaged)")
        elif 'PROTECTED' in res['verdict']:
            print(f"  {family_name}: ❌ A3 refuted (Heritage protects)")
        else:
            print(f"  {family_name}: ⚠️ Unclear ({res['verdict']})")

if total_tested > 0:
    print(f"\nA3 Pattern: {a3_pattern_count}/{total_tested} new families")
    if a3_pattern_count == total_tested:
        print(f"\n🎯 A3 CLAIM STRENGTHENED: Now 5 families, 3 architectures!")
    elif a3_pattern_count > 0:
        print(f"\n⚠️ A3 CLAIM PARTIALLY EXTENDED: {a3_pattern_count} new families confirm pattern")
    else:
        print(f"\n❌ A3 CLAIM NOT EXTENDED: No new families show pattern")

In [ ]:
# Cell 9: Visualization

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

families = list(all_results.keys())
colors = {'early': '#e74c3c', 'middle': '#3498db', 'late': '#2ecc71'}

# Plot 1: Early Layer Fragility Comparison
ax1 = axes[0]
early_deltas = []
family_labels = []
for family_name, res in all_results.items():
    if res['status'] == 'success':
        early_deltas.append(res['deltas']['early']['percent'])
        family_labels.append(f"{family_name}\n({res['detected_arch']})")

if early_deltas:
    bars = ax1.bar(family_labels, early_deltas, color='#e74c3c', alpha=0.7)
    ax1.axhline(y=0, color='black', linestyle='--')
    ax1.axhline(y=30, color='orange', linestyle=':', label='A3 threshold (30%)')
    ax1.set_ylabel('Early Layer Δ (%)')
    ax1.set_title('Early Layer Fragility Change\n(Instruct vs Base)')
    ax1.legend()
    
    # Add value labels
    for bar, val in zip(bars, early_deltas):
        ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, 
                f'{val:+.0f}%', ha='center', va='bottom')

# Plot 2: All Regions Comparison
ax2 = axes[1]
x = np.arange(len(family_labels))
width = 0.25

for i, (family_name, res) in enumerate(all_results.items()):
    if res['status'] == 'success':
        if i == 0:
            early_vals = [res['deltas']['early']['percent']]
            mid_vals = [res['deltas']['middle']['percent']]
            late_vals = [res['deltas']['late']['percent']]
        else:
            early_vals.append(res['deltas']['early']['percent'])
            mid_vals.append(res['deltas']['middle']['percent'])
            late_vals.append(res['deltas']['late']['percent'])

if family_labels:
    ax2.bar(x - width, early_vals, width, label='Early', color=colors['early'], alpha=0.7)
    ax2.bar(x, mid_vals, width, label='Middle', color=colors['middle'], alpha=0.7)
    ax2.bar(x + width, late_vals, width, label='Late', color=colors['late'], alpha=0.7)
    ax2.axhline(y=0, color='black', linestyle='--')
    ax2.set_xticks(x)
    ax2.set_xticklabels(family_labels)
    ax2.set_ylabel('Δ Fragility (%)')
    ax2.set_title('Layer-Wise Fragility Change\nby Architecture')
    ax2.legend()

# Plot 3: Architecture Comparison (including prior GQA)
ax3 = axes[2]

# Combine prior GQA with new results
all_families = ['Gemma-27B', 'LLaMA-3.1-8B', 'Qwen2-7B']
all_early = [150, 51, 117]  # Prior GQA results
all_archs = ['GQA', 'GQA', 'GQA']

for family_name, res in all_results.items():
    if res['status'] == 'success':
        all_families.append(family_name)
        all_early.append(res['deltas']['early']['percent'])
        all_archs.append(res['detected_arch'])

# Color by architecture
arch_colors = {'GQA': '#3498db', 'MHA': '#e74c3c', 'MQA': '#2ecc71'}
bar_colors = [arch_colors.get(a, '#95a5a6') for a in all_archs]

bars = ax3.bar(range(len(all_families)), all_early, color=bar_colors, alpha=0.7)
ax3.axhline(y=0, color='black', linestyle='--')
ax3.axhline(y=30, color='orange', linestyle=':', label='A3 threshold')
ax3.set_xticks(range(len(all_families)))
ax3.set_xticklabels(all_families, rotation=45, ha='right')
ax3.set_ylabel('Early Layer Δ (%)')
ax3.set_title('Cross-Architecture Comparison\n(All 5 Families)')

# Add architecture legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=c, label=a, alpha=0.7) for a, c in arch_colors.items()]
ax3.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()

fig_path = f'figures/E04b_heritage_expansion_{TIMESTAMP}.png'
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFigure saved: {fig_path}")

In [ ]:
# Cell 10: Save Results

def convert_to_native(obj):
    if isinstance(obj, dict):
        return {k: convert_to_native(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [convert_to_native(v) for v in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_to_native(v) for v in obj)
    elif isinstance(obj, (np.bool_, np.integer)):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    else:
        return obj

filename = f'results/E04b_heritage_expansion_{TIMESTAMP}.json'

# Count successful tests
success_count = sum(1 for r in all_results.values() if r['status'] == 'success')
a3_count = sum(1 for r in all_results.values() if r['status'] == 'success' and 'DAMAGED' in r.get('verdict', ''))

output = {
    'experiment': 'E04b-Heritage-Expansion',
    'purpose': 'Extend A3 claim to MHA + MQA architectures',
    'timestamp': TIMESTAMP,
    
    # E11-v3 Methodology Block (REQUIRED)
    'methodology': {
        'standard': 'E11-v3',
        'seeds': ACTIVE_CONFIG['seeds'],
        'max_length': MAX_LENGTH,
        'dtype': str(DTYPE),
        'prompt_md5': actual_md5,
        'prompt_md5_verified': prompts_ok,
        'num_prompts': ACTIVE_CONFIG['prompts_subset'],
        'total_prompts': len(STANDARD_PROMPTS),
        'prompt_set': 'Standard-10',
        'noise_levels': ACTIVE_CONFIG['noise_levels'],
        'use_chat_template_base': USE_CHAT_TEMPLATE_BASE,
        'use_chat_template_instruct': USE_CHAT_TEMPLATE_INSTRUCT,
        'attention_masked': True
    },
    
    'phase': 'Phase 1 (Screening)' if ACTIVE_CONFIG == PHASE1_CONFIG else 'Phase 2 (E11-v3 Full)',
    'model_pairs': {k: {'base': v['base'], 'instruct': v['instruct'], 'expected_arch': v['expected_arch']} 
                   for k, v in MODEL_PAIRS.items()},
    'results': convert_to_native(all_results),
    'summary': {
        'total_tested': len(all_results),
        'successful': success_count,
        'a3_pattern_found': a3_count,
        'architectures_tested': list(set(r.get('detected_arch', r.get('expected_arch', '?')) 
                                        for r in all_results.values())),
        'conclusion': 'A3 extended' if a3_count == success_count and success_count > 0 else 
                     'Partial extension' if a3_count > 0 else 'No extension'
    },
    'prior_gqa_reference': {
        'Gemma-27B': {'early_delta': '+150%', 'source': 'E11-Indra'},
        'LLaMA-3.1-8B': {'early_delta': '+51%', 'source': 'E04-LLaMA31'},
        'Qwen2-7B': {'early_delta': '+117%', 'source': 'E04-Qwen'}
    }
}

with open(filename, 'w') as f:
    json.dump(output, f, indent=2)

print(f"Results saved: {filename}")
print(f"\nSummary:")
print(f"  Total tested: {output['summary']['total_tested']}")
print(f"  Successful: {output['summary']['successful']}")
print(f"  A3 pattern: {output['summary']['a3_pattern_found']}")
print(f"  Conclusion: {output['summary']['conclusion']}")


In [ ]:
# Cell 11: Auto-Download Results

import glob
import shutil

def auto_download_results():
    try:
        from google.colab import files
    except ImportError:
        print('Not in Colab - skipping auto-download')
        return
    
    print('=' * 60)
    print('AUTO-DOWNLOADING RESULTS...')
    print('=' * 60)
    
    json_files = glob.glob('results/*.json') + glob.glob('figures/*.json')
    png_files = glob.glob('results/*.png') + glob.glob('figures/*.png')
    all_files = json_files + png_files
    
    if not all_files:
        print('WARNING: No result files found!')
        return
    
    print(f'Found {len(all_files)} files')
    
    import os
    zip_name = f'E04b_heritage_expansion_results_{TIMESTAMP}'
    
    os.makedirs('download_package', exist_ok=True)
    for f in all_files:
        shutil.copy(f, 'download_package/')
    
    shutil.make_archive(zip_name, 'zip', 'download_package')
    print(f'Downloading: {zip_name}.zip')
    files.download(f'{zip_name}.zip')
    print('DOWNLOAD COMPLETE!')

auto_download_results()

---

## Summary: E04b Heritage Expansion

### Purpose
Extend A3 claim (Heritage > Scale) from 3 GQA families to 5 families across 3 architectures.

### New Architectures Tested

| Architecture | Family | What it tests |
|-------------|--------|---------------|
| **MHA** | LLaMA-2-7B | Classic attention (all heads independent) |
| **MQA** | Falcon-7B | Single KV head (extreme sharing) |

### Expected Outcomes

If A3 pattern (Early damaged, Middle/Late immune) holds:
- **A3 upgraded to A-Tier** with 5 families, 3 architectures
- Pattern is **architecture-invariant** (fundamental to RLHF)

If pattern breaks:
- A3 becomes **architecture-conditional**
- MHA/MQA may have different fragility dynamics

### Phase Progression

1. **Phase 1 (This notebook):** Quick screening (1 seed, 3 noise levels)
2. **Phase 2 (If signal found):** Full validation (3 seeds, 6 noise levels)

---

*Paper 4: Behavioral Sink Dynamics*  
*E04b: Heritage Expansion*